In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
import numpy as np
import math
import wandb
from sklearn.preprocessing import MinMaxScaler

In [18]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Hesaplamalar için kullanılacak cihaz: {device}')

wandb.login()

Hesaplamalar için kullanılacak cihaz: cuda


True

In [19]:
def create_sequences(data, lookback_window):
    X, y = [], []
    for i in range(len(data) - lookback_window):
        feature = data[i:(i + lookback_window)]
        target = data[i + lookback_window]
        X.append(feature)
        y.append(target)
    return torch.FloatTensor(np.array(X)), torch.FloatTensor(np.array(y))

In [20]:
df = pd.read_csv('genel_sunucu_yuku_tum_metrikler.csv', index_col='timestamp')
df.index = pd.to_datetime(df.index, unit='s')
ts_data = df[['avg_min_cpu', 'avg_max_cpu', 'avg_avg_cpu']].values
scaler = MinMaxScaler(feature_range=(0, 1))
ts_data_scaled = scaler.fit_transform(ts_data)
train_size = int(len(ts_data_scaled) * 0.8)
train_data, test_data = ts_data_scaled[:train_size], ts_data_scaled[train_size:]

lookback = 12

# PyTorch Dataset ve DataLoader'ları oluştur
X_train, y_train = create_sequences(train_data, lookback)
X_test, y_test = create_sequences(test_data, lookback)

train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

print("Veri setleri ve DataLoader'lar hazır.")


Veri setleri ve DataLoader'lar hazır.


In [21]:
# PositionalEncoding ve TransformerModel sınıfları
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super(PositionalEncoding, self).__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        return x + self.pe[:x.size(0), :]



In [22]:
class TransformerModel(nn.Module):
    def __init__(self, input_dim=3, d_model=64, nhead=4, num_encoder_layers=2, dim_feedforward=128, dropout=0.1):
        super(TransformerModel, self).__init__()
        self.d_model = d_model
        self.encoder = nn.Linear(input_dim, d_model)
        self.pos_encoder = PositionalEncoding(d_model)
        encoder_layers = nn.TransformerEncoderLayer(d_model, nhead, dim_feedforward, dropout, batch_first=True)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, num_encoder_layers)
        self.decoder = nn.Linear(d_model, input_dim) # Çıktı boyutu girdiyle aynı (3)

    def forward(self, src):
        src = self.encoder(src) * math.sqrt(self.d_model)
        src = self.pos_encoder(src)
        output = self.transformer_encoder(src)
        output = self.decoder(output[:, -1, :])
        return output


In [23]:
def train_and_evaluate_transformer(config):
    #  WandB Deneyini Başlat
    run = wandb.init(project="bites-cpu-forecasting", config=config, reinit=True)
    
    #  DataLoader'ları Oluştur
    train_loader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=config['batch_size'])
    
    #  Modeli Oluştur
    model = TransformerModel(
        input_dim=3,
        d_model=config['d_model'],
        nhead=config['nhead'],
        num_encoder_layers=config['num_encoder_layers'],
        dim_feedforward=config['dim_feedforward'],
        dropout=config['dropout']
    ).to(device)
    
    #  Optimizer ve Loss Fonksiyonunu Tanımla
    optimizer = optim.Adam(model.parameters(), lr=config['lr'])
    criterion = nn.MSELoss() # Regresyon için MSELoss
    
    #  Modeli ve Gradyanları İzle
    wandb.watch(model, criterion, log="all", log_freq=10)

    #  Eğitim Döngüsü
    print(f"\n--- Deney Başladı: {run.name} ---")
    for epoch in range(config['epochs']):
        # --- Eğitim Adımı ---
        model.train()
        total_train_loss = 0
        for batch_x, batch_y in train_loader:
            batch_x, batch_y = batch_x.to(device), batch_y.to(device)
            optimizer.zero_grad()
            outputs = model(batch_x)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()
            total_train_loss += loss.item()
        
        avg_train_loss = total_train_loss / len(train_loader)

        # --- Değerlendirme Adımı ---
        model.eval()
        total_test_loss = 0
        with torch.no_grad():
            for batch_x, batch_y in test_loader:
                batch_x, batch_y = batch_x.to(device), batch_y.to(device)
                outputs = model(batch_x)
                total_test_loss += criterion(outputs, batch_y).item()
        
        avg_test_loss = total_test_loss / len(test_loader)
        
        #  Her Epoch Sonunda WandB'ye Logla
        wandb.log({
            "epoch": epoch, 
            "train_loss": avg_train_loss,
            "test_loss": avg_test_loss
        })
        print(f"Epoch {epoch+1}/{config['epochs']} | Train Loss: {avg_train_loss:.6f} | Test Loss: {avg_test_loss:.6f}")

    #  Modeli Kaydet ve Deneyi Bitir
    model_path = f"model_path/transformer_{run.name}.pt"
    torch.save(model.state_dict(), model_path)
    wandb.save(model_path)
    run.finish()


In [24]:
# Denenecek farklı ve daha çeşitli hiperparametre setleri
configs = [
    # 1. Mevcut Deneyler (Karşılaştırma için)
    {'d_model': 64, 'nhead': 4, 'num_encoder_layers': 2, 'dim_feedforward': 128, 'dropout': 0.1},
    {'d_model': 128, 'nhead': 8, 'num_encoder_layers': 3, 'dim_feedforward': 256, 'dropout': 0.2},
    
    # 2. Daha Küçük ve Hızlı Bir Model
    {'d_model': 32, 'nhead': 4, 'num_encoder_layers': 1, 'dim_feedforward': 64, 'dropout': 0.1},
    
    # 3. Daha Derin Bir Model
    {'d_model': 64, 'nhead': 8, 'num_encoder_layers': 4, 'dim_feedforward': 128, 'dropout': 0.2},
    
    # 4. Daha Geniş Bir Model (Daha fazla 'beyin gücü')
    {'d_model': 256, 'nhead': 8, 'num_encoder_layers': 2, 'dim_feedforward': 512, 'dropout': 0.2},
    
    # 5. Farklı Öğrenme Oranları ile Deneyler
    {'d_model': 128, 'nhead': 8, 'num_encoder_layers': 3, 'dim_feedforward': 256, 'dropout': 0.2, 'lr': 5e-5}, # Daha yavaş öğrenme
    {'d_model': 64, 'nhead': 4, 'num_encoder_layers': 2, 'dim_feedforward': 128, 'dropout': 0.1, 'lr': 5e-4}, # Daha hızlı öğrenme
    
    # 6. Daha Yüksek Dropout ile Deney
    {'d_model': 128, 'nhead': 8, 'num_encoder_layers': 3, 'dim_feedforward': 256, 'dropout': 0.4, 'lr': 1e-4} # Ezberlemeye karşı daha agresif
]

# Tüm deneyler için ortak olan sabit parametreler
base_config = {
    'batch_size': 64,
    'lr': 1e-4, # Her config içinde belirtilmezse bu değer kullanılır
    'epochs': 20
}

# WandB'ye giriş yaptığınızdan emin olun: terminalde 'wandb login'

# Tüm konfigürasyonları sırayla dene
for cfg in configs:
    print(f"\n--- Yeni Deney Başlatılıyor ---")
    print(f"Parametreler: {cfg}")
    
    # Sabit ve denenecek parametreleri birleştir
    full_config = {**base_config, **cfg}
    
    # Ana eğitim fonksiyonunu bu konfigürasyon ile çağır
    train_and_evaluate_transformer(full_config)
    
    # Deneyler arası GPU belleğini temizleyerek olası hataları önle
    torch.cuda.empty_cache()



--- Yeni Deney Başlatılıyor ---
Parametreler: {'d_model': 64, 'nhead': 4, 'num_encoder_layers': 2, 'dim_feedforward': 128, 'dropout': 0.1}



--- Deney Başladı: serene-night-26 ---
Epoch 1/20 | Train Loss: 0.062013 | Test Loss: 0.009092
Epoch 2/20 | Train Loss: 0.015208 | Test Loss: 0.004868
Epoch 3/20 | Train Loss: 0.010616 | Test Loss: 0.003685
Epoch 4/20 | Train Loss: 0.008232 | Test Loss: 0.002924
Epoch 5/20 | Train Loss: 0.006856 | Test Loss: 0.002675
Epoch 6/20 | Train Loss: 0.006050 | Test Loss: 0.002492
Epoch 7/20 | Train Loss: 0.005251 | Test Loss: 0.002416
Epoch 8/20 | Train Loss: 0.004861 | Test Loss: 0.002349
Epoch 9/20 | Train Loss: 0.004512 | Test Loss: 0.002296
Epoch 10/20 | Train Loss: 0.004166 | Test Loss: 0.002255
Epoch 11/20 | Train Loss: 0.003949 | Test Loss: 0.002200
Epoch 12/20 | Train Loss: 0.003691 | Test Loss: 0.002162
Epoch 13/20 | Train Loss: 0.003531 | Test Loss: 0.002193
Epoch 14/20 | Train Loss: 0.003455 | Test Loss: 0.002190
Epoch 15/20 | Train Loss: 0.003274 | Test Loss: 0.002159
Epoch 16/20 | Train Loss: 0.003163 | Test Loss: 0.002198
Epoch 17/20 | Train Loss: 0.003112 | Test Loss: 0.002125


epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
test_loss,█▄▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_loss,█▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,19
test_loss,0.00217
train_loss,0.00288



--- Yeni Deney Başlatılıyor ---
Parametreler: {'d_model': 128, 'nhead': 8, 'num_encoder_layers': 3, 'dim_feedforward': 256, 'dropout': 0.2}



--- Deney Başladı: hopeful-forest-27 ---
Epoch 1/20 | Train Loss: 0.055160 | Test Loss: 0.004093
Epoch 2/20 | Train Loss: 0.021419 | Test Loss: 0.002826
Epoch 3/20 | Train Loss: 0.014689 | Test Loss: 0.002386
Epoch 4/20 | Train Loss: 0.011192 | Test Loss: 0.002506
Epoch 5/20 | Train Loss: 0.008951 | Test Loss: 0.002361
Epoch 6/20 | Train Loss: 0.007413 | Test Loss: 0.002248
Epoch 7/20 | Train Loss: 0.006479 | Test Loss: 0.002284
Epoch 8/20 | Train Loss: 0.005738 | Test Loss: 0.002340
Epoch 9/20 | Train Loss: 0.005275 | Test Loss: 0.002139
Epoch 10/20 | Train Loss: 0.004826 | Test Loss: 0.002173
Epoch 11/20 | Train Loss: 0.004673 | Test Loss: 0.002095
Epoch 12/20 | Train Loss: 0.004367 | Test Loss: 0.002129
Epoch 13/20 | Train Loss: 0.004071 | Test Loss: 0.002165
Epoch 14/20 | Train Loss: 0.003996 | Test Loss: 0.002135
Epoch 15/20 | Train Loss: 0.003751 | Test Loss: 0.002144
Epoch 16/20 | Train Loss: 0.003664 | Test Loss: 0.002088
Epoch 17/20 | Train Loss: 0.003501 | Test Loss: 0.00204

...


--- Deney Başladı: mild-planet-32 ---
Epoch 1/20 | Train Loss: 0.037003 | Test Loss: 0.004249
Epoch 2/20 | Train Loss: 0.007403 | Test Loss: 0.002694
Epoch 3/20 | Train Loss: 0.005092 | Test Loss: 0.002298
Epoch 4/20 | Train Loss: 0.004074 | Test Loss: 0.002255
Epoch 5/20 | Train Loss: 0.003533 | Test Loss: 0.002334
Epoch 6/20 | Train Loss: 0.003197 | Test Loss: 0.002148
Epoch 7/20 | Train Loss: 0.003002 | Test Loss: 0.002123
Epoch 8/20 | Train Loss: 0.002875 | Test Loss: 0.002614
Epoch 9/20 | Train Loss: 0.002741 | Test Loss: 0.002102
Epoch 10/20 | Train Loss: 0.002700 | Test Loss: 0.002081
Epoch 11/20 | Train Loss: 0.002615 | Test Loss: 0.002076
Epoch 12/20 | Train Loss: 0.002586 | Test Loss: 0.002009
Epoch 13/20 | Train Loss: 0.002517 | Test Loss: 0.002061
Epoch 14/20 | Train Loss: 0.002465 | Test Loss: 0.001996
Epoch 15/20 | Train Loss: 0.002442 | Test Loss: 0.002055
Epoch 16/20 | Train Loss: 0.002421 | Test Loss: 0.002135
Epoch 17/20 | Train Loss: 0.002351 | Test Loss: 0.002256
E

epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
test_loss,█▃▂▂▂▂▁▃▁▁▁▁▁▁▁▁▂▁▁▁
train_loss,█▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,19
test_loss,0.00198
train_loss,0.00233



--- Yeni Deney Başlatılıyor ---
Parametreler: {'d_model': 128, 'nhead': 8, 'num_encoder_layers': 3, 'dim_feedforward': 256, 'dropout': 0.4, 'lr': 0.0001}



--- Deney Başladı: avid-armadillo-33 ---
Epoch 1/20 | Train Loss: 0.073286 | Test Loss: 0.008214
Epoch 2/20 | Train Loss: 0.031709 | Test Loss: 0.003466
Epoch 3/20 | Train Loss: 0.020327 | Test Loss: 0.002680
Epoch 4/20 | Train Loss: 0.014514 | Test Loss: 0.002985
Epoch 5/20 | Train Loss: 0.011230 | Test Loss: 0.002416
Epoch 6/20 | Train Loss: 0.009647 | Test Loss: 0.002396
Epoch 7/20 | Train Loss: 0.008172 | Test Loss: 0.002415
Epoch 8/20 | Train Loss: 0.007364 | Test Loss: 0.002390
Epoch 9/20 | Train Loss: 0.006732 | Test Loss: 0.002607
Epoch 10/20 | Train Loss: 0.006067 | Test Loss: 0.002821
Epoch 11/20 | Train Loss: 0.005659 | Test Loss: 0.002415
Epoch 12/20 | Train Loss: 0.005238 | Test Loss: 0.002392
Epoch 13/20 | Train Loss: 0.004915 | Test Loss: 0.002449
Epoch 14/20 | Train Loss: 0.004574 | Test Loss: 0.002342
Epoch 15/20 | Train Loss: 0.004525 | Test Loss: 0.002338
Epoch 16/20 | Train Loss: 0.004214 | Test Loss: 0.002517
Epoch 17/20 | Train Loss: 0.004072 | Test Loss: 0.00259

epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
test_loss,█▂▁▂▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▂
train_loss,█▄▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,19
test_loss,0.00278
train_loss,0.0036


In [25]:
import wandb
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import plotly.graph_objects as go
import numpy as np

def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / (y_true + 1e-8))) * 100

# --- 1. Adım: En İyi Deneyin Hiperparametrelerini ve Model Dosyasını Belirle ---

# WandB arayüzünden en iyi Transformer deneyini bularak bu bilgileri doldurmalısın.
# Optuna'nın bulduğu sonuçları veya deney listesindeki en düşük test_loss'a sahip olanı kullanabilirsin.
# Aşağıdakiler bir önceki Optuna sonucuna göre örnektir:
best_run_config = {
    'd_model': 64, 
    'nhead': 4, 
    'num_encoder_layers': 2, 
    'dim_feedforward': 128,
    'dropout': 0.1, # Eğitim fonksiyonundaki dropout değeri ile eşleşmeli
    'lr': 0.0005,
    'batch_size': 64,
    'epochs':20
}
# En iyi denemenin kaydettiği model dosyasının adı.
# Örnek: "transformer_mild-planet-32.pt". Kendi deneyinin adıyla değiştir.
best_model_filename = "model_path/transformer_mild-planet-32.pt" 

print(f"En iyi model dosyası: '{best_model_filename}'")
print("Kullanılan hiperparametreler:", best_run_config)


# --- 2. Adım: Model Mimarisi İskeletini Oluştur ---

# En iyi hiperparametrelerle modelin boş bir kopyasını oluştur
final_model = TransformerModel(
    input_dim=3,
    d_model=best_run_config['d_model'],
    nhead=best_run_config['nhead'],
    num_encoder_layers=best_run_config['num_encoder_layers'],
    dim_feedforward=best_run_config['dim_feedforward'],
    dropout=best_run_config['dropout']
).to(device)


# --- 3. Adım: Kayıtlı Ağırlıkları Modele Yükle ---

try:
    # Kayıtlı ağırlıkları bu iskeletin içine yükle
    final_model.load_state_dict(torch.load(best_model_filename))
    print(f"\n'{best_model_filename}' dosyasından en iyi modelin ağırlıkları başarıyla yüklendi.")
except FileNotFoundError:
    print(f"\nHATA: '{best_model_filename}' dosyası bulunamadı.")
    print("Lütfen dosya adını doğru olduğundan veya dosyanın bu notebook ile aynı klasörde olduğundan emin ol.")


# --- 4. Adım: Yüklenen Model ile Nihai Tahmin ve Değerlendirmeyi Yap ---

final_model.eval()
test_predictions = []
with torch.no_grad():
    for seq, label in zip(X_test, y_test):
        seq_gpu = seq.unsqueeze(0).to(device)
        y_pred = final_model(seq_gpu)
        test_predictions.append(y_pred.squeeze(0).cpu().numpy())

predictions_original_scale = scaler.inverse_transform(np.array(test_predictions))
y_test_original_scale = scaler.inverse_transform(y_test.numpy())

# --- 5. Adım: Nihai Metrikleri ve Grafiği Hazırla ---

target_columns = ['avg_min_cpu', 'avg_max_cpu', 'avg_avg_cpu']
final_metrics = {}

print("\n--- Nihai Modelin Performans Metrikleri ---")
for i, column in enumerate(target_columns):
    y_true_col = y_test_original_scale[:, i]
    y_pred_col = predictions_original_scale[:, i]
    
    mae = mean_absolute_error(y_true_col, y_pred_col)
    rmse = np.sqrt(mean_squared_error(y_true_col, y_pred_col))
    r2 = r2_score(y_true_col, y_pred_col)
    mape = mean_absolute_percentage_error(y_true_col, y_pred_col)
    
    final_metrics[column] = {'MAE': mae, 'RMSE': rmse, 'R2': r2, 'MAPE': mape}
    
    print(f"\n--- Metrik: '{column}' ---")
    print(f"  MAE: {mae:.2f}, RMSE: {rmse:.2f}, R²: {r2:.2f}, MAPE: {mape:.2f}%")

# Nihai grafiği oluştur
final_fig = go.Figure()
for i, column in enumerate(target_columns):
    final_fig.add_trace(go.Scatter(x=df.index[train_size+lookback:], y=y_test_original_scale[:, i], mode='lines', name=f'Gerçek {column}'))
    final_fig.add_trace(go.Scatter(x=df.index[train_size+lookback:], y=predictions_original_scale[:, i], mode='lines', name=f'Tahmin {column}', line=dict(dash='dash')))
final_fig.update_layout(title='En İyi Transformer Modelinin Nihai Tahmin Performansı', xaxis_title='Zaman', yaxis_title='Ortalama CPU Yükü (%)')
final_fig.show()

# --- 6. Adım: Nihai Sonuçları WandB'ye Logla ---

print("\nNihai sonuçlar WandB'ye loglanıyor...")
run = wandb.init(project="bites-cpu-forecasting", name="Final_Evaluation_Best_Transformer", config=best_run_config)

log_dict = {}
for col_name, metrics in final_metrics.items():
    for metric_name, value in metrics.items():
        log_dict[f"{col_name}_{metric_name}"] = value

log_dict["Final Tahmin Grafiği"] = final_fig
wandb.log(log_dict)

run.finish()
print("Loglama tamamlandı.")


En iyi model dosyası: 'model_path/transformer_mild-planet-32.pt'
Kullanılan hiperparametreler: {'d_model': 64, 'nhead': 4, 'num_encoder_layers': 2, 'dim_feedforward': 128, 'dropout': 0.1, 'lr': 0.0005, 'batch_size': 64, 'epochs': 20}

'model_path/transformer_mild-planet-32.pt' dosyasından en iyi modelin ağırlıkları başarıyla yüklendi.

--- Nihai Modelin Performans Metrikleri ---

--- Metrik: 'avg_min_cpu' ---
  MAE: 0.06, RMSE: 0.08, R²: 0.87, MAPE: 0.95%

--- Metrik: 'avg_max_cpu' ---
  MAE: 0.24, RMSE: 0.30, R²: 0.73, MAPE: 1.20%

--- Metrik: 'avg_avg_cpu' ---
  MAE: 0.11, RMSE: 0.14, R²: 0.82, MAPE: 1.16%



Nihai sonuçlar WandB'ye loglanıyor...


avg_avg_cpu_MAE,▁
avg_avg_cpu_MAPE,▁
avg_avg_cpu_R2,▁
avg_avg_cpu_RMSE,▁
avg_max_cpu_MAE,▁
avg_max_cpu_MAPE,▁
avg_max_cpu_R2,▁
avg_max_cpu_RMSE,▁
avg_min_cpu_MAE,▁
avg_min_cpu_MAPE,▁
avg_min_cpu_R2,▁


Loglama tamamlandı.
